## Setup

### Imports

In [1]:
import logging
import imp
import dataset
import utils
import proxynca
import net
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import json
import random
from utils import JSONEncoder, json_dumps
from datetime import datetime as dt
import pandas as pd
import os

### Args Class

Class containing the parameters for training the model

In [2]:
class args():
    dataset = 'food'
    config = 'config1.json'
    sz_embedding = 64 #size of the embedding that is appendet to inceptionv2
    sz_batch = 32 #number of samples per batch
    nb_epochs = 40
    gpu_id = 0
    nb_workers = 4
    with_nmi = True  #turn calculations for nmi on or off turn off for sop
    scaling_x = 3.0 #scaling factor for the normalized embeddings
    scaling_p = 3.0 #scaling factor for the normalized proxies
    lr_proxynca = 1.0 #learning rate for proxynca
    log_filename = (f'''{dataset}-{dt.now().strftime("%Y%m%d-%H%M%S")}''')
    results_filename = f'{dataset}-results.csv'
    torch_version = str(torch.__version__)
    edition = 0
    seed = 0

### Seed Everything

In [3]:
def seed_everything(args = args):
    seed = args.seed
    if seed != -1:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    else:
        print('not seeded')


### Choose Device

In [4]:
torch.cuda.set_device(args.gpu_id)

### Setup the config

In [5]:
def setup_config(args = args):
    config = utils.load_config(args.config)
    config['criterion']['args']['scaling_x'] = args.scaling_x
    config['criterion']['args']['scaling_p'] = args.scaling_p
    config['opt']['args']['proxynca']['lr'] = args.lr_proxynca
    return config

### DataLoader

In [6]:
def load_tr(config = setup_config(), args = args):
    dl_tr = torch.utils.data.DataLoader(
        dataset.load(name = args.dataset,
                     root = config['dataset'][args.dataset]['root'],
                     classes = config['dataset'][args.dataset]['classes']['train'],
                     transform = dataset.utils.make_transform(**config['transform_parameters'])
                    ),
        batch_size = args.sz_batch,
        shuffle = True,
        num_workers = args.nb_workers,
        drop_last = True,
        pin_memory = True
    )
    return dl_tr

def load_ev(config = setup_config(), args = args):
    dl_ev = torch.utils.data.DataLoader(
        dataset.load(
            name = args.dataset,
            root = config['dataset'][args.dataset]['root'],
            classes = config['dataset'][args.dataset]['classes']['eval'],
            transform = dataset.utils.make_transform(
                **config['transform_parameters'],
                is_train = False)
        ),
        batch_size = args.sz_batch,
        shuffle = False,
        num_workers = args.nb_workers,
        pin_memory = True
    )
    return dl_ev


### Network

In [7]:
def setup_model(args = args):
    model = net.bn_inception(pretrained = True)
    net.embed(model, sz_embedding = args.sz_embedding)
    model = model.cuda()
    return model


### Criterion

This function initializes the training class

In [8]:
def setup_criterion(config = setup_config(), args = args, dl_tr = load_tr()):
    criterion = proxynca.ProxyNCA(
        nb_classes = dl_tr.dataset.nb_classes(),
        sz_embedding = args.sz_embedding,
        **config['criterion']['args']).cuda()
    return criterion

### Optimizer

In [9]:
def setup_opt(config = setup_config(), model = setup_model(), criterion = setup_criterion()):
    opt = config['opt']['type'](
        [
            { # inception parameters, excluding embedding layer
                **{'params': list(
                    set(
                        model.parameters()
                    ).difference(
                        set(model.embedding_layer.parameters())
                    )
                )},
                **config['opt']['args']['backbone']
            },
            { # embedding parameters
                **{'params': model.embedding_layer.parameters()},
                **config['opt']['args']['embedding']
            },
            { # proxy nca parameters
                **{'params': criterion.parameters()},
                **config['opt']['args']['proxynca']
            }
        ],
        **config['opt']['args']['base']
    )
    return opt


### Scheduler

In [10]:
def setup_scheduler(config = setup_config(), opt = setup_opt()):
    scheduler = config['lr_scheduler']['type'](
        opt, **config['lr_scheduler']['args'])
    return scheduler

### Logging

In [11]:
def setup_logging(args = args):
    imp.reload(logging)
    logging.basicConfig(
        format = "%(asctime)s %(message)s",
        level = logging.INFO,
        handlers = [
            logging.FileHandler("{0}/{1}.log".format('log', args.log_filename)),
            logging.StreamHandler()
        ]
    )

    logging.info("Training parameters: {}".format(vars(args)))
    logging.info("Training for {} epochs".format(args.nb_epochs))

## Training

In [12]:
def train_and_test(args = args):
    #set up new parameters
    seed_everything(args)
    config = setup_config(args)
    dl_tr = load_tr(config, args)
    dl_ev = load_ev(config, args)
    model = setup_model(args = args)
    criterion = setup_criterion(config = config, args = args, dl_tr = load_tr())
    opt = setup_opt(config = config, model = model, criterion = criterion)
    scheduler = setup_scheduler(config = config, opt = opt)
    setup_logging(args = args)
    
    if args.with_nmi == True:
        df = pd.DataFrame(columns = ['epoch', 'r@1', 'r@2', 'r@4', 'r@8', 'NMI'])
    else:
        df = pd.DataFrame(columns = ['epoch', 'r@1', 'r@2', 'r@4','r@8'])
    
    losses = []
    t1 = time.time()
    logging.info("**Evaluating initial model.**")
    with torch.no_grad():
        utils.evaluate(model, dl_ev, with_nmi = args.with_nmi)

    for e in range(0, args.nb_epochs):
        if e!=0:
            scheduler.step()
        time_per_epoch_1 = time.time()
        losses_per_epoch = []
        for x,y, _ in dl_tr:
            opt.zero_grad()
            m = model(x.cuda())
            loss = criterion(m, y.cuda())
            loss.backward()

#             torch.nn.utils.clip_grad_value_(model.parameters(), 10)
            
            losses_per_epoch.append(loss.data.cpu().numpy())
            opt.step()

        time_per_epoch_2 = time.time()
        losses.append(np.mean(losses_per_epoch[-20:]))
        logging.info(
            "Epoch: {}, loss: {:.3f}, time (seconds): {:.2f}.".format(
                e,
                losses[-1],
                time_per_epoch_2 - time_per_epoch_1))
        with torch.no_grad():
            logging.info("**Evaluating.**")
            recall = utils.evaluate(model, dl_ev, with_nmi = args.with_nmi) #variable name not accurate
            # append results of current epoch to df
            if args.with_nmi == True:
                lst = recall[0].copy()
                lst.append(recall[1])
                lst.insert(0,e)
                df_epoch = pd.DataFrame([lst], columns = ['epoch', 'r@1', 'r@2', 'r@4','r@8', 'NMI'])
            else:
                lst = recall.copy()
                lst.insert(0,e)
                df_epoch = pd.DataFrame([lst], columns = ['epoch', 'r@1', 'r@2', 'r@4', 'r@8'])
            df = pd.concat([df,df_epoch])
            model.losses = losses
            model.current_epoch = e

    t2 = time.time()
    logging.info("Total training time (minutes): {:.2f}.".format((t2 - t1) / 60))
    return df

## Grid Search

In [ ]:
# lists of each parameter to search
seeds = [0]
lrs = [1]
scaling_xs = [1.0,3.0,8.0]
scaling_ps = [1.0,3.0,8.0]
sz_embs = [64]
sz_batches = [32,128]
eds = [0]

results = {}
if os.path.exists(args.results_filename):
    results_df = pd.read_csv(args.results_filename)
    index = 0
else:
    results_df = pd.DataFrame(columns = ['index','epoch', 'r@1', 'r@2', 'r@4', 'r@8', 'NMI',
                                         'lr','scl_x','scl_p','sz_emb','seed', 'edition',
                                         'batch', 'torch version'])
    index = 0
for lr in lrs:
    args.lr_proxynca = lr
    for scl_x in scaling_xs:
        args.scaling_x = scl_x
        for scl_p in scaling_ps:
            args.scaling_p = scl_p
            for sz_emb in sz_embs:
                args.sz_embeddings = sz_emb
                for seed in seeds:
                    args.seed = seed
                    for ed in eds:
                        args.edition = ed
                        for sz_batch in sz_batches:
                            args.sz_batch = sz_batch
                            if (results_df[(results_df.lr == lr) & (results_df.scl_x == scl_x)
                                               & (results_df.scl_p == scl_p)
                                               & (results_df.sz_emb == sz_emb)
                                               & (results_df.seed == seed)
                                               & (results_df.edition == ed)
                                               & (results_df['torch version'] == args.torch_version)
                                               & (results_df['batch'] == sz_batch)]
                                .shape[0] == 0):
                                if results_df['index'].shape[0] > 0:
                                    index = results_df['index'].max() + 1
                                print(index)
                                res_df = train_and_test()
                                res_df['lr'] = lr
                                res_df['scl_x'] = scl_x
                                res_df['scl_p'] = scl_p
                                res_df['index'] = index
                                res_df['sz_emb'] = sz_emb
                                res_df['seed'] = seed
                                res_df['edition'] = ed
                                res_df['batch'] = sz_batch
                                res_df['torch version'] = args.torch_version
                                results_df = pd.concat([results_df, res_df])
                                results_df.to_csv(args.results_filename, index = False)
                                index+=1
                            else:
                                print(f'skipped version:{index}. It was already done.')
                                index+=1

skipped version:0. It was already done.
skipped version:1. It was already done.
skipped version:2. It was already done.
skipped version:3. It was already done.
skipped version:4. It was already done.
skipped version:5. It was already done.
skipped version:6. It was already done.
skipped version:7. It was already done.
skipped version:8. It was already done.
skipped version:9. It was already done.
skipped version:10. It was already done.
skipped version:11. It was already done.
45


2020-09-19 14:49:42,501 Training parameters: {'__module__': '__main__', 'dataset': 'food', 'config': 'config1.json', 'sz_embedding': 64, 'sz_batch': 32, 'nb_epochs': 40, 'gpu_id': 0, 'nb_workers': 4, 'with_nmi': True, 'scaling_x': 8.0, 'scaling_p': 1.0, 'lr_proxynca': 1, 'log_filename': 'food-20200919-144937', 'results_filename': 'food-results.csv', 'torch_version': '1.1.0', 'edition': 0, 'seed': 0, '__dict__': <attribute '__dict__' of 'args' objects>, '__weakref__': <attribute '__weakref__' of 'args' objects>, '__doc__': None, 'sz_embeddings': 64}
2020-09-19 14:49:42,502 Training for 40 epochs
2020-09-19 14:49:42,503 **Evaluating initial model.**
2020-09-19 14:51:45,610 NMI: 24.435
2020-09-19 14:53:21,747 R@1 : 42.006
2020-09-19 14:53:22,473 R@2 : 50.544
2020-09-19 14:53:23,200 R@4 : 59.936
2020-09-19 14:53:23,924 R@8 : 69.261
2020-09-19 14:56:01,068 Epoch: 0, loss: 2.305, time (seconds): 157.14.
2020-09-19 14:56:01,069 **Evaluating.**
2020-09-19 14:58:24,501 NMI: 28.771
2020-09-19 14

2020-09-19 17:26:01,146 R@8 : 70.167
2020-09-19 17:28:31,508 Epoch: 25, loss: 1.156, time (seconds): 150.36.
2020-09-19 17:28:31,509 **Evaluating.**
2020-09-19 17:30:36,839 NMI: 25.303
2020-09-19 17:32:08,964 R@1 : 44.651
2020-09-19 17:32:09,695 R@2 : 52.847
2020-09-19 17:32:10,419 R@4 : 61.751
2020-09-19 17:32:11,142 R@8 : 70.306
2020-09-19 17:34:39,230 Epoch: 26, loss: 1.100, time (seconds): 148.09.
2020-09-19 17:34:39,231 **Evaluating.**
2020-09-19 17:37:10,557 NMI: 24.671
2020-09-19 17:38:40,936 R@1 : 43.797
2020-09-19 17:38:41,659 R@2 : 52.339
2020-09-19 17:38:42,382 R@4 : 61.094
2020-09-19 17:38:43,104 R@8 : 69.798
2020-09-19 17:41:15,418 Epoch: 27, loss: 1.123, time (seconds): 152.31.
2020-09-19 17:41:15,419 **Evaluating.**
2020-09-19 17:43:33,351 NMI: 25.397
2020-09-19 17:45:03,709 R@1 : 44.485
2020-09-19 17:45:04,437 R@2 : 52.832
2020-09-19 17:45:05,158 R@4 : 61.597
2020-09-19 17:45:05,883 R@8 : 70.219
2020-09-19 17:47:30,905 Epoch: 28, loss: 1.137, time (seconds): 145.02.
202

46


2020-09-19 18:56:47,039 Training parameters: {'__module__': '__main__', 'dataset': 'food', 'config': 'config1.json', 'sz_embedding': 64, 'sz_batch': 128, 'nb_epochs': 40, 'gpu_id': 0, 'nb_workers': 4, 'with_nmi': True, 'scaling_x': 8.0, 'scaling_p': 1.0, 'lr_proxynca': 1, 'log_filename': 'food-20200919-144937', 'results_filename': 'food-results.csv', 'torch_version': '1.1.0', 'edition': 0, 'seed': 0, '__dict__': <attribute '__dict__' of 'args' objects>, '__weakref__': <attribute '__weakref__' of 'args' objects>, '__doc__': None, 'sz_embeddings': 64}
2020-09-19 18:56:47,039 Training for 40 epochs
2020-09-19 18:56:47,051 **Evaluating initial model.**
2020-09-19 18:58:27,643 NMI: 24.435
2020-09-19 19:00:02,494 R@1 : 42.006
2020-09-19 19:00:03,231 R@2 : 50.544
2020-09-19 19:00:03,951 R@4 : 59.936
2020-09-19 19:00:04,671 R@8 : 69.261
2020-09-19 19:01:57,324 Epoch: 0, loss: 2.388, time (seconds): 112.65.
2020-09-19 19:01:57,325 **Evaluating.**
2020-09-19 19:03:53,592 NMI: 28.865
2020-09-19 1

2020-09-19 21:27:05,741 R@8 : 73.363
2020-09-19 21:29:21,905 Epoch: 25, loss: 1.140, time (seconds): 136.16.
2020-09-19 21:29:21,905 **Evaluating.**
2020-09-19 21:31:40,090 NMI: 28.629
2020-09-19 21:33:19,490 R@1 : 47.180
2020-09-19 21:33:20,226 R@2 : 55.802
2020-09-19 21:33:20,961 R@4 : 64.520
2020-09-19 21:33:21,714 R@8 : 73.108
2020-09-19 21:35:38,828 Epoch: 26, loss: 1.110, time (seconds): 137.11.
2020-09-19 21:35:38,829 **Evaluating.**
2020-09-19 21:37:47,645 NMI: 28.116
2020-09-19 21:39:27,663 R@1 : 46.793
2020-09-19 21:39:28,408 R@2 : 55.416
2020-09-19 21:39:29,158 R@4 : 64.326
2020-09-19 21:39:29,915 R@8 : 73.079
2020-09-19 21:41:47,074 Epoch: 27, loss: 1.152, time (seconds): 137.16.
2020-09-19 21:41:47,075 **Evaluating.**
2020-09-19 21:44:09,398 NMI: 27.905
2020-09-19 21:45:49,033 R@1 : 46.669
2020-09-19 21:45:49,781 R@2 : 55.503
2020-09-19 21:45:50,544 R@4 : 64.419
2020-09-19 21:45:51,303 R@8 : 72.902
2020-09-19 21:48:10,492 Epoch: 28, loss: 1.106, time (seconds): 139.19.
202

47


2020-09-19 23:01:57,913 Training parameters: {'__module__': '__main__', 'dataset': 'food', 'config': 'config1.json', 'sz_embedding': 64, 'sz_batch': 32, 'nb_epochs': 40, 'gpu_id': 0, 'nb_workers': 4, 'with_nmi': True, 'scaling_x': 8.0, 'scaling_p': 3.0, 'lr_proxynca': 1, 'log_filename': 'food-20200919-144937', 'results_filename': 'food-results.csv', 'torch_version': '1.1.0', 'edition': 0, 'seed': 0, '__dict__': <attribute '__dict__' of 'args' objects>, '__weakref__': <attribute '__weakref__' of 'args' objects>, '__doc__': None, 'sz_embeddings': 64}
2020-09-19 23:01:57,913 Training for 40 epochs
2020-09-19 23:01:57,915 **Evaluating initial model.**
2020-09-19 23:04:14,121 NMI: 24.435
2020-09-19 23:05:52,912 R@1 : 42.006
2020-09-19 23:05:53,651 R@2 : 50.544
2020-09-19 23:05:54,386 R@4 : 59.936
2020-09-19 23:05:55,119 R@8 : 69.261
2020-09-19 23:08:37,392 Epoch: 0, loss: 2.590, time (seconds): 162.27.
2020-09-19 23:08:37,393 **Evaluating.**
2020-09-19 23:11:10,179 NMI: 26.629
2020-09-19 23